In [ ]:
import numpy as np
import csv
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer


In [ ]:
path = 'CapstoneAgHandsTrainingData.csv'
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

In [ ]:
dataset = loadtxt(path, delimiter=',')
print(dataset[1])
print(np.shape(dataset[1]))
# split into input (X) and output (y) variables
X = dataset[:,0:63]
y = dataset[:,63]
print(np.shape(X))
print(np.shape(y))


[-3.64892185e-04  8.65041673e-01 -2.39167832e-08  7.70429522e-02
  8.53903890e-01 -2.16601621e-02  1.61481366e-01  7.80207753e-01
 -2.58291494e-02  2.23874435e-01  7.06370473e-01 -2.62705851e-02
  2.65073180e-01  6.59262598e-01 -2.60710586e-02  1.81350440e-01
  7.09594488e-01 -1.41459005e-02  2.50110418e-01  6.64438725e-01
 -2.59353705e-02  2.85198122e-01  6.53899789e-01 -3.82892489e-02
  3.10844600e-01  6.34108126e-01 -4.83197756e-02  1.47838145e-01
  6.76919878e-01 -7.58573087e-03  1.99503526e-01  6.47702456e-01
 -2.01434828e-02  1.83849901e-01  6.88274205e-01 -2.73974091e-02
  1.64306879e-01  7.15717971e-01 -3.06640696e-02  1.17306069e-01
  6.60287082e-01 -2.88019259e-03  1.67126089e-01  6.29279792e-01
 -1.54732578e-02  1.52461231e-01  6.71338320e-01 -9.01640765e-03
  1.36284426e-01  7.03534007e-01 -1.71150931e-03  9.26678106e-02
  6.53549075e-01  1.81023119e-04  1.32571459e-01  6.20021880e-01
 -6.07311586e-03  1.21956877e-01  6.57139182e-01  3.55407689e-03
  1.09315798e-01  6.88482

In [ ]:
#Validation/training Data Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(y_train)

(1908, 63)
(477, 63)
(1908,)
(477,)
[0. 1. 0. ... 0. 1. 0.]


In [ ]:
#One hot encode outputs
y_test = LabelBinarizer().fit_transform(y_test)
y_train = LabelBinarizer().fit_transform(y_train)
print(y_train.shape)
print(y_test.shape)

(1908, 3)
(477, 3)


In [ ]:

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import *

In [ ]:
def get_model():
    x = x_in = Input(shape=(63,), name="input")
    x = Dense(63, name="d1", activation="relu")(x)
    x = Dense(126, name="d2", activation="relu")(x)
    x = Dense(252, name="d3", activation="relu")(x)
    x = Dropout(0.2, name="d5")(x)
    x = Dense(126, name="d4", activation="relu")(x)
    
    x = Dense(3, name="output")(x)
    x = Activation("softmax", name="s1")(x)

    m = Model(inputs=x_in, outputs=x)
    m.summary()
    
    return m

adam = Adam(lr=0.01)

model = get_model()
model.compile(loss="binary_crossentropy", optimizer=adam, metrics=["acc"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 63)]              0         
                                                                 
 d1 (Dense)                  (None, 63)                4032      
                                                                 
 d2 (Dense)                  (None, 126)               8064      
                                                                 
 d3 (Dense)                  (None, 252)               32004     
                                                                 
 d5 (Dropout)                (None, 252)               0         
                                                                 
 d4 (Dense)                  (None, 126)               31878     
                                                                 
 output (Dense)              (None, 3)                 381   

In [ ]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
60/60 [==============================] - 0s 3ms/step - loss: 0.4075 - acc: 0.7123
Epoch 2/5
60/60 [==============================] - 0s 3ms/step - loss: 0.2479 - acc: 0.8485
Epoch 3/5
60/60 [==============================] - 0s 3ms/step - loss: 0.1696 - acc: 0.9004
Epoch 4/5
60/60 [==============================] - 0s 3ms/step - loss: 0.1687 - acc: 0.9025
Epoch 5/5
60/60 [==============================] - 0s 3ms/step - loss: 0.1655 - acc: 0.9083


In [ ]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

15/15 - 0s - loss: 0.4464 - acc: 0.6855 - 157ms/epoch - 10ms/step

Test accuracy: 0.6855345964431763


In [ ]:
model.save('CapstoneAgModelV2.h5')

In [ ]:
model.load_weights('CapstoneAgModelV2.h5')

In [ ]:
loss, acc = model.evaluate(X_test, y_test, verbose=1)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

15/15 [==============================] - 0s 4ms/step - loss: 0.1074 - acc: 0.9455
Restored model, accuracy: 94.55%


In [ ]:
np.shape(X_test)

(477, 63)

In [ ]:
from tensorflow import keras 
reconstructed_model = keras.models.load_model("CapstoneAgModelV2.h5") 
print(np.shape(model.predict(X_test)))

(477, 3)


In [ ]:
np.shape(X_test)
print(X_test[1])

[ 9.65266705e-01  5.63700616e-01 -3.83935117e-07  9.23147440e-01
  5.78281879e-01 -8.23608693e-03  8.84075701e-01  5.76195896e-01
 -1.29550574e-02  8.54434013e-01  5.68514824e-01 -1.78192779e-02
  8.29347372e-01  5.65971017e-01 -2.32084636e-02  8.77182364e-01
  5.21876633e-01 -1.43131102e-02  8.71679127e-01  5.54831028e-01
 -2.93293465e-02  8.74034762e-01  5.83457530e-01 -4.05804291e-02
  8.77190650e-01  6.07500792e-01 -4.76681069e-02  8.98343861e-01
  5.11421561e-01 -1.79208107e-02  8.96478772e-01  5.58504939e-01
 -3.16483900e-02  9.00571108e-01  5.93599379e-01 -4.01539095e-02
  9.05248284e-01  6.22447729e-01 -4.66516092e-02  9.23221946e-01
  5.07980049e-01 -2.27934215e-02  9.20908689e-01  5.54965317e-01
 -3.63235772e-02  9.22826886e-01  5.89477539e-01 -4.02321294e-02
  9.25944626e-01  6.18509233e-01 -4.25178185e-02  9.48954582e-01
  5.08575976e-01 -2.83125602e-02  9.46817636e-01  5.49011469e-01
 -4.02556285e-02  9.45463955e-01  5.78302801e-01 -4.31830250e-02
  9.43926334e-01  6.01261

In [ ]:
np.shape(np.zeros(63))


(63,)